# The detailed demonstration of the tropical rainfall diagnostic for low-resolution data:

# Calculation of histograms

### Content


1. [The load of packages, modules, and data](#1-load-of-packages-modules-and-data)

2. [Calculation of histogram of precipitation/tropical precipitation](#2-calculation-of-histogram-of-precipitationtropicalprecipitation)

    2.1. [with continuous  uniform binning](#21-with-continuous-uniform-binning)

    2.2  [with non-uniform (log-spaced) binning](#22-with-non-uniform-log-spaced-binning)

    2.3. [in the lazy (or delayed) mode](#23-in-the-lazy-or-delayed-mode)
    
    2.4. [with non-default tropical latitude band](#24-with-non-default-latitude-band)

    2.5  [with a specific time band](#25-with-a-specific-time-band)

    2.6. [with weights (weights=reader.grid_area)](#26-with-weights-weightsreadergrid_area)
 

3. [Loading the histogram to/from storage](#3-loading-the-histogram-tofrom-storage)

[Go to the end of file](#the-end)

We split the demonstration of the histogram calculation for low-resolution and high-resolution data into two notebooks. 

The reason for that is that in the current diagnostic deliverable, the histogram calculations for low-resolution data have more features. Namely, the description of the functionality is following:

|  | Low-resolution | High-resolution |
| --- | --- | --- |
| name of function | histogram_lowres  | histogram |
| uniform binning | + | + |
| non-uniform (log-spaced) binning | + | - |
| lazy (or delayed) mode | + | - |
|  weights (weights=reader.grid_area) | + | - |

In the next deliverable of the diagnostic, we will add all necessary features for high-resolution data (like non-uniform binning).

#

## 1. The load of packages, modules, and data

In [1]:
import sys
from aqua import Reader
sys.path.insert(0, '../../')
from tropical_rainfall import Tropical_Rainfall

OpenBLAS blas_thread_init: pthread_create failed for thread 121 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 122 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 123 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 124 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 125 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 126 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1

KeyboardInterrupt: 

In [ ]:
diag = Tropical_Rainfall()
path_to_mean = diag.path_to_netcdf+'mean/'

In [ ]:
if diag.tools.get_machine()==lumi:
    reader  = Reader(model="IFS", exp="historical-1990-devcon", source="lra-r100-monthly") 
    ifs_1990_devcon = reader.retrieve(var=["tprate"])
    
    reader                  = Reader(model="IFS", exp="control-1950-devcon", source="lra-r100-monthly")
    control_1950_devcon     = reader.retrieve(var=["tprate"])
else:
    reader  = Reader(model="IFS", exp="tco2559-ng5-cycle3", source="lra-r100-monthly")
    ifs_tco2559_ng5    = reader.retrieve(var=["tprate"])

    reader                  = Reader(model="IFS", exp="tco1279-orca025-cycle3", source="lra-r100-monthly")
    ifs_tco1279_orca025     = reader.retrieve(var=["tprate"])

2023-10-20 17:34:50 :: Reader :: WARNING  -> You are asking for var ['tprate'] which is already fixed from ['tp'].
2023-10-20 17:34:50 :: Reader :: WARNING  -> It would be safer to run with fix=False
2023-10-20 17:34:50 :: Reader :: WARNING  -> You are asking for var ['tprate'] which is already fixed from ['tp'].
2023-10-20 17:34:50 :: Reader :: WARNING  -> Would be safer to run with fix=False
2023-10-20 17:34:50 :: eccodes :: WARNING  -> ShortName tprate have multiple grib codes associated: ['260048', '172228']
2023-10-20 17:34:50 :: eccodes :: WARNING  -> AQUA will take the first so that tprate -> 260048, please set up a correct fix if this does not look right
2023-10-20 17:34:54 :: Reader :: WARNING  -> You are asking for var ['tprate'] which is already fixed from ['tp'].
2023-10-20 17:34:54 :: Reader :: WARNING  -> It would be safer to run with fix=False
2023-10-20 17:34:54 :: Reader :: WARNING  -> You are asking for var ['tprate'] which is already fixed from ['tp'].
2023-10-20 17:

In [7]:
if diag.tools.get_machine()==lumi:
    lra_data = {
                'historical_1990': {'data': ifs_1990_devcon, 'label': 'ifs_historical-1990-devcon'},
                'control_1950': {'data': control_1950_devcon, 'label': 'ifs_control-1950-devcon'},            
            }
else:
    lra_data = {
                'ifs_tco2559': {'data': ifs_tco2559_ng5, 'label': 'ifs_tco2559'},    
                'ifs_tco1279': {'data': ifs_tco1279_orca025, 'label': 'ifs_tco1279'},     
            }

## 2. Calculation of histogram of tropical precipitation


### 2.1. with continuous uniform binning 

##### We can perform calculations of the histograms with uniform binning by  initializing the following attributes of diagnostic: 
 - `num_of_bins`
 - `first_edge`
 - `width_of_bin`
 
 ##### In this case, all the bins in the histogram will have the width of bins.

##### We calculate the histogram simply by function:

In [ ]:
histograms = {}
for key, value in lra_data.items():
    histograms[key] = diag.histogram(value["data"], 
                                     new_unit=new_unit,  path_to_histogram=path_to_netcdf, name_of_file=value["label"])

##### The output of the histogram function is xarray.Dataset, which has two coordinates 
- `center_of_bin`:   the center of each bin
- `width`:           width of each bin
##### and contains three variables:
- `counts`:       the number of observations that fall into each bin
- `frequency`:    the number of cases in each bin, normalized by the total number of counts. The sum of the frequencies equals 1.
- `pdf`:          the number of cases in each bin, normalized by the total number of counts and width of each bin. 

##### local and global attributes. Local attributes contain the information about the time and space grid for which diagnostic performed the calculations:
- `time_band`:    the value of time of the first and last element in the dataset and the frequency of the time grid
- `lat_band`:     the maximum and minimum values of the tropical latitude band and the frequency of the latitude grid
- `lon_band`:     the maximum and minimum values of the longitude and the frequency of the longitude grid

##### Global attribute `history` contains the information about when the histogram was calculated and values of `time_band`, `lat_band`, and `lon_band`.

### 2.2.  with non-uniform (log-spaced) binning  

##### If you want to perform the calculation for non-uniform binning, use the `bins` attribute of the diagnostic instead of  `num_of_bins`, `first_edge`, and `width_of_bin`.  

In [5]:
bins = [1.00000000e-09, 1.63789371e-09, 2.68269580e-09, 4.39397056e-09,
       7.19685673e-09, 1.17876863e-08, 1.93069773e-08, 3.16227766e-08,
       5.17947468e-08, 8.48342898e-08, 1.38949549e-07, 2.27584593e-07,
       3.72759372e-07, 6.10540230e-07, 1.00000000e-06]

In [6]:
diag = Tropical_Rainfall(bins=bins)

In [ ]:
histograms_logspaced = {}
for key, value in lra_data.items():
    histograms_logspaced[key] = diag.histogram_lowres(value["data"], 
                                     new_unit=new_unit,  path_to_histogram=path_to_netcdf, name_of_file=value["label"]+"_logspaced")

### 2.3. in the lazy (or delayed) mode

##### To perform calculations in the so-called lazy mode, use the flag `lazy` in the histogram function. 

In [8]:
diag = Tropical_Rainfall(trop_lat=15,  num_of_bins = 5000, first_edge = 0, loglevel='warning')
diag.width_of_bin = diag.precipitation_rate_units_converter(0.2, old_unit='mm/day', new_unit=new_unit)


In [8]:
histograms_lazy = {}
for key, value in lra_data.items():
    histograms_lazy[key] = diag.histogram_lowres(value["data"], new_unit=new_unit,
                                     lazy=True)

/LUMI_TYKKY_PuGxLUy/miniconda/envs/env1/lib/python3.11/site-packages/dask/array/core.py:3470: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
/LUMI_TYKKY_PuGxLUy/miniconda/envs/env1/lib/python3.11/site-packages/dask/array/core.py:3470: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


##### `Note`: In the case of lazy calculation, the function's output will be different:  the xarray will contain only non-computed counts. If you want to add frequency and pdf variables to the histogram Dataset, apply the following function `add_frequency_and_pdf` (but only when you are actually ready to compute the histogram).

In [19]:
for key,value in lra_data.items():
    histograms_lazy[key]  = diag.add_frequency_and_pdf(tprate_dataset=histograms_lazy[key])

### 2.4. with non-default latitude band 

##### The tropical latitude band, by default, is the following

In [11]:
diag.trop_lat

10

##### You can modify the tropical latitude band as 

In [12]:
diag.trop_lat = 20
diag.trop_lat

20

##### You can also modify the latitude band with the use of an argument `trop_lat` of the histogram function 

In [22]:
histograms_trop30 = {}
for key, value in lra_data.items():
    histograms_trop30[key] = diag.histogram(value["data"], trop_lat=30, new_unit=new_unit)

### 2.5. with a specific time band 

##### You can specify `s_time` and `f_time` as integers (starting and final times). For example, 

In [28]:
diag = Tropical_Rainfall(trop_lat=15,  num_of_bins = 5000, first_edge = 0, loglevel='warning', s_time = 0, f_time = 5)
diag.width_of_bin = diag.precipitation_rate_units_converter(0.2, old_unit='mm/day', new_unit=new_unit)


In [29]:
histograms_timeband = {}
for key, value in lra_data.items():
    histograms_timeband[key] = diag.histogram(value["data"], new_unit=new_unit)

##### Also, you can specify `s_time` and `f_time` as strings. For example, 

In [16]:
diag.s_time = '2020:01'
diag.f_time ='2020/03/20/12'

##### or

In [17]:
diag.s_time = '2020'
diag.f_time ='2020.03.20'

##### There is the possibility of specifying only the year band or only the months' band. For example, we can select June, July, and August in a whole dataset as

In [18]:
diag.s_month = 3
diag.f_month = 6 

In order to evaluate the original dataset, we reset the time band at the full time range available in the dataset:

In [19]:
diag.s_time = None
diag.f_time = None
diag.s_month = None
diag.f_month = None

### 2.6. with weights (weights=reader.grid_area)

##### The function provides the opportunity to calculate the histogram with weights. Compared to standard methods, such computations are `high-speed` because they are based on `dask_histogram` package.

In [30]:
histograms_weighted = {}
for key, value in lra_data.items():
    histograms_weighted[key] = diag.histogram(value["data"], new_unit=new_unit, weights=reader.grid_area)

##### You can also parse only the `bins` attribute of the class  instead of the specification of `num_of_bins`, `first_edge`, and `width_of_bin`. 

##### But such calculations would be much `slower`! (at around 50 times)

In [22]:
diag = Tropical_Rainfall(bins = bins)

In [ ]:
histograms_weighted = {}
for key, value in lra_data.items():
    histograms_weighted[key] = diag.histogram_lowres(value["data"], new_unit=new_unit, weights=reader.grid_area)

#

## 3. Loading the histogram to/from storage

#### Saving the histogram in storage. 

##### `Notice`: the name of the file will be unique and depends on the time band 

In [ ]:
histograms = {}
for key, value in lra_data.items():
    histograms[key] = diag.histogram(value["data"], 
                                     new_unit=new_unit,  path_to_histogram=path_to_netcdf, name_of_file=value["label"])

#### Loading/opening the histogram from the storage 

##### The information about how to open and merge the list of histograms, see the notebook  `diagnostic_vs_streaming.ipynb`

In [31]:
if lumi:
    filename = path_to_netcdf+'trop_rainfall_ifs_control-1950-devcon_histogram_0-0_1000-0_5000_1950-01-01T00_1959-12-01T00_M.nc'
    diag.open_dataset(path_to_netcdf = filename)

[Back to the top of file](#structure)

##### The end